# Bibliotecas

In [ ]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import pickle
import os
import asyncio
import nest_asyncio
import random
import pandas as pd


# Criação de um template do excel para manipulação de dados
O template criado é um exemplo da estrutura de dados para armazenar membros, trilhas, cursos e suas respectivas associações

In [ ]:
#tome muito cuidado para não sobrescresver o arquivo populado
NOME_TEMPLATE = 'TEMPLATE.xlsx'

membros_feadev = pd.DataFrame({
        'id_membro': [0, 1, 2],
        'nome': ['admin', 'Tiago Toledo', 'Rogério Ceni'],
        'email': ['email-falso-admin-dev@gmail.com', 'ttduarte@usp.br', 'rogerio-ceni@gmail.com'],
        'conta_github': ['admin-feadev-github', 'Tiago745', 'Rogerio-Ceni-Github'],
        'conta_datacamp': ['', '', ''], #provavelmente será uma url para cursos finalizados, ainda precisa ser revisado
        'xp_datacamp': ['', '', ''],
        'ativo': [1, 1, 0]
})

trilhas = pd.DataFrame({
        'id_trilha': [0, 1, 2], #chave/id/pk
        'nome_trilha': ['Associate Data Scientist in Python', 'Associate Data Engineer in SQL',  'Capacitação 2025 - Básico'],
        'url': ["https://app.datacamp.com/learn/career-tracks/associate-data-scientist-in-python","https://app.datacamp.com/learn/career-tracks/associate-data-engineer-in-sql", ''],
        'tipo_trilha': [0,0,1] # 0-> trilha do datacamp || 1-> trilha personalizada (default deve ser 0, 1 somente quando for personalizado ou criado in-app)
        })

cursos = pd.DataFrame({
    'id_curso': [0, 1, 2],
    'nome_curso': ['Introduction to Python', 'Intermediate Python', 'SQL'],
    'duracao' : [4, 4, 4],
    'url' : ['', '', '']
})

eventos = pd.DataFrame({
    'id_evento': [0],
    'nome_evento': ['QuantiConnect 2025 - Dia 1'],
    'tipo_evento_id': ['2'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'tipo_evento_nome': ['Feira'], #PK/CHAVE ESTRANGEIRA DE Tipos_de_Eventos
    'descricao': ['Primeiro dia do QuantiConnect 2025, evento organizado pela Feadev com foco em inovação quantitativa, networking e palestras.'],
    'data_inicio' : [pd.to_datetime('2025-05-10 10:00')],
    'data_fim' : [pd.to_datetime('2025-05-10 17:00')],
})

tipos_de_eventos = pd.DataFrame({
    'id_tipo_de_evento': [0, 1, 2],
    'nome_tipo_de_evento': ['Reunião', 'Apresentação', 'Feira'],
})



#DataFrames Associativos
trilhas_tem_cursos = pd.DataFrame({
    'id_trilha' : [0, 0, 1, 2, 2, 2],
    'id_curso' : [0, 1, 2, 0, 1, 2],
    'ordem_curso' : [0, 1, 0, 1, 2, 0], #ordem em que os cursos devem ser assistidos dentro de cada trilha
    'data_final_para_assistir': ['', '', '','20/06/2023', '20/06/2023', '23/06/2023'], #cursos de trilhas do datacamp nao tem data-final, apenas trilhas personalizadas
    'obrigatoriedade_curso': [1, 1, 1, 1, 1, 1] #o curso precisa ser obrigatoriamente assistido ou é opcional, 1->Obrigatório 0->Não obrigatório/opcional
})

membro_feadev_faz_trilhas = pd.DataFrame({
    'id_membro' : [1],
    'id_trilha' : [2],
    'data_inicio' : ['15/06/2025'],
    'data_fim' : ['20/06/2025'],
    'finalizado' : [True]
})

membro_feadev_faz_cursos = pd.DataFrame({
    'id_membro' : [1, 1, 1],
    'id_curso' : [0,1,2],
    'data_inicio' : ['15/06/2025', '17/06/2025', '20/06/2025'],
    'data_fim' : ['15/06/2025', '21/06/2025', '21/06/2025'],
    'finalizado' : [True, True, True]
})

membro_feadev_participa_eventos = pd.DataFrame({
    'id_membro' : [0],
    'id_evento' : [0],
    'presença' : [1], #0 = Ausente, 1 = Presente
})

#Uniao de trilhas com cursos
resultado = trilhas.merge(trilhas_tem_cursos, on='id_trilha') \
                   .merge(cursos, on='id_curso') \
                   #[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso']]  # remove colunas extras

resultado_ordenado = resultado.sort_values(by=['id_trilha', 'ordem_curso']) #ordena por trilha e depois por ordem que deve assistir cada curso, facilita a leitura

print(resultado_ordenado[['id_trilha', 'nome_trilha', 'id_curso', 'nome_curso', 'ordem_curso']])

#limpando as linhas de lorem ipsum em trilhas e cursos
trilhas = trilhas[0:0]
cursos = cursos[0:0]

# Caminho do arquivo Excel que será salvo
arquivo_excel = 'dados_trilhas.xlsx'

# Usando o ExcelWriter para salvar cada DataFrame em uma aba diferente
with pd.ExcelWriter(NOME_TEMPLATE, engine='openpyxl') as writer:
    trilhas.to_excel(writer, sheet_name='trilhas', index=False)
    cursos.to_excel(writer, sheet_name='cursos', index=False)
    eventos.to_excel(writer, sheet_name='eventos', index=False)
    tipos_de_eventos.to_excel(writer, sheet_name='tipos_de_eventos', index=False)
    trilhas_tem_cursos.to_excel(writer, sheet_name='trilhas_tem_cursos', index=False)
    membros_feadev.to_excel(writer, sheet_name='membros_feadev', index=False)
    membro_feadev_faz_trilhas.to_excel(writer, sheet_name='membro_feadev_faz_trilhas', index=False)
    membro_feadev_faz_cursos.to_excel(writer, sheet_name='membro_feadev_faz_cursos', index=False)
    membro_feadev_participa_eventos.to_excel(writer, sheet_name='membro_feadev_participa_eventos', index=False)

print("✅ Arquivo Excel salvo com sucesso!")


   id_trilha                         nome_trilha  id_curso  \
0          0  Associate Data Scientist in Python         0   
1          0  Associate Data Scientist in Python         1   
2          1      Associate Data Engineer in SQL         2   
5          2           Capacitação 2025 - Básico         2   
3          2           Capacitação 2025 - Básico         0   
4          2           Capacitação 2025 - Básico         1   

               nome_curso  ordem_curso  
0  Introduction to Python            0  
1     Intermediate Python            1  
2                     SQL            0  
5                     SQL            0  
3  Introduction to Python            1  
4     Intermediate Python            2  
✅ Arquivo Excel salvo com sucesso!


# Atualizar cursos e trilhas do datacamp
Inicalmente atualiza as sheets de [Trilhas] e de [Cursos], depois disso faz a associação entre os ids de trilhas e cursos, gerando a sheet [Trilhas_tem_Cursos]

In [ ]:
#nome do arquivo em que está contido o banco de dados do maedev
NOME_ARQUIVO = 'arquivo.xlsx'
PASTA_TRILHAS = '../trilhas'

#Recupera o nome de cada arquivo dentro da subpasta trilhas_atualizadas
LISTA_DE_TRILHAS_PARA_LER = [f for f in os.listdir(PASTA_TRILHAS) if os.path.isfile(os.path.join(PASTA_TRILHAS, f))]



def atualizar_trilhas_com_ids(trilhas_atualizadas_datacamp, maedev_df, col_nome_trilha_atualizada="Trilha", col_df_maedev=['id_trilha', 'nome_trilha', 'url', 'tipo_trilha']):
    """
    Adiciona novas trilhas únicas de `trilhas_atualizados_datacamp` em `maedev_df`, com id incremental.
    
    Parâmetros:
        trilhas_atualizados_datacamp (pd.DataFrame): DataFrame com os cursos e suas trilhas.
        maedev_df (pd.DataFrame): DataFrame com trilhas já cadastradas.
        col_nome_trilha_atualizada (str): Nome da trilha no trilhass_atualizadas_datacamp que contém o nome das trilhas.
        col_df_maedev (list): Lista com os nomes das colunas na ordem: [id, nome, url, tipo].

    Retorna:
        pd.DataFrame: O DataFrame `maedev_df` atualizado com novas trilhas e IDs únicos.
    """
    url_padrao_trilha = 'https://app.datacamp.com/learn/career-tracks/'
    
    # Padroniza os nomes das trilhas
    trilhas_atualizadas_datacamp[col_nome_trilha_atualizada] = trilhas_atualizadas_datacamp[col_nome_trilha_atualizada].astype(str).str.strip()
    maedev_df[col_df_maedev[1]] = maedev_df[col_df_maedev[1]].astype(str).str.strip()

    # ID inicial baseado no maior ID atual
    ultimo_id = maedev_df[col_df_maedev[0]].max() if not maedev_df.empty else -1
    novas_linhas = []



     # Itera cada coluna unica em cursos atualizados(apenas combinacoes unicas), depois acessa o valor nas colunas nome_curso e Duração
    for _, row in trilhas_atualizadas_datacamp[[col_nome_trilha_atualizada, 'tipo_trilha']].drop_duplicates().iterrows():
        trilha = row[col_nome_trilha_atualizada]
        tipo_da_trilha = row['tipo_trilha'] #0-> trilha oficial do datacamp, 1-> trilha personalizada

        #Se essa trilha não está inclusa no maedev, adicona ela a lista de novos trilhas
        if trilha not in maedev_df[col_df_maedev[1]].values:
            ultimo_id += 1
            novas_linhas.append({
                col_df_maedev[0]: ultimo_id, #id
                col_df_maedev[1]: trilha, #nome da trilha
                col_df_maedev[2]: url_padrao_trilha + "-".join(trilha.lower().split()), #url da trilha
                col_df_maedev[3]: tipo_da_trilha
            })
            print(f"✅ Novo curso adicionado: {trilha} (id = {ultimo_id})")

    # Adiciona as novas trilhas, se houver
    if novas_linhas:
        df_novos = pd.DataFrame(novas_linhas)
        maedev_df = pd.concat([maedev_df, df_novos], ignore_index=True)
        print("🎯 Atualização concluída com sucesso.")
    else:
        print("ℹ️ Nenhuma nova trilha encontrada.")

    return maedev_df

def atualizar_cursos_com_ids(cursos_atualizados_datacamp, maedev_df, col_nome_curso_atualizada="Curso", col_df_maedev=['id_curso', 'nome_curso', 'duracao', 'url']):
    """
    Adiciona novas trilhas únicas de `cursos_atualizados_datacamp` em `maedev_df`, com id incremental.
    
    Parâmetros:
        cursos_atualizados_datacamp (pd.DataFrame): DataFrame com os cursos e suas trilhas.
        maedev_df (pd.DataFrame): DataFrame com trilhas já cadastradas.
        col_nome_curso_atualizada (str): Nome da coluna no cursos_atualizados_datacamp que contém o nome das trilhas.
        col_df_maedev (list): Lista com os nomes das colunas na ordem: [id, nome, url, tipo].

    Retorna:
        pd.DataFrame: O DataFrame `maedev_df` atualizado com novas trilhas e IDs únicos.
    """
    url_padrao_trilha = 'https://app.datacamp.com/learn/courses/'
    
    # Padroniza os nomes dos cursos
    cursos_atualizados_datacamp[col_nome_curso_atualizada] = cursos_atualizados_datacamp[col_nome_curso_atualizada].astype(str).str.strip()
    maedev_df[col_df_maedev[1]] = maedev_df[col_df_maedev[1]].astype(str).str.strip()

    # ID inicial baseado no maior ID atual
    ultimo_id = maedev_df[col_df_maedev[0]].max() if not maedev_df.empty else -1
    novas_linhas = []


    # Itera cada coluna unica em cursos atualizados(apenas combinacoes unicas), depois acessa o valor nas colunas nome_curso e Duração
    for _, row in cursos_atualizados_datacamp[[col_nome_curso_atualizada, 'Duração']].drop_duplicates().iterrows():
        curso = row[col_nome_curso_atualizada]
        duracao = row['Duração']

        #Se esse curso não está incluso no maedev, adicona ele a lista de novos cursos
        if curso not in maedev_df[col_df_maedev[1]].values:
            ultimo_id += 1
            novas_linhas.append({
                col_df_maedev[0]: ultimo_id,
                col_df_maedev[1]: curso,
                col_df_maedev[2]: duracao,
                col_df_maedev[3]: url_padrao_trilha + "-".join(curso.lower().split()),
            })
            print(f"✅ Novo curso adicionado: {curso} (id = {ultimo_id})")

    # Adiciona os novos cursos, se houver
    if novas_linhas:
        df_novos = pd.DataFrame(novas_linhas)
        maedev_df = pd.concat([maedev_df, df_novos], ignore_index=True)
        print("🎯 Atualização concluída com sucesso.")
    else:
        print("ℹ️ Nenhum novo curso encontrado.")

    return maedev_df

#Para associar as trilhas com os cursos/ gerar associações
def gerar_trilhas_tem_cursos(caminho_arquivo, pasta_trilhas):
    """
    Gera a sheet 'Trilhas_tem_Cursos' no arquivo Excel especificado, associando cada trilha aos cursos correspondentes.

    A função percorre todos os arquivos .xlsx dentro da pasta fornecida, assumindo que cada arquivo representa uma trilha
    (coluna 'Trilha') com os cursos correspondentes (coluna 'Curso'). Com base nesses nomes, ela busca os respectivos 
    `id_trilha` e `id_curso` nas sheets 'Trilhas' e 'Cursos' do arquivo principal, criando a associação com campos extras 
    padrão.

    A nova sheet gerada contém as seguintes colunas:
        - id_trilha (int)
        - id_curso (int)
        - data_final_para_assistir (str): sempre vazio ('')
        - obrigatoriedade_curso (int): sempre 1

    Parâmetros:
    ----------
    caminho_arquivo : str
        Caminho para o arquivo Excel principal (ex: 'arquivo.xlsx') que contém as sheets 'Trilhas' e 'Cursos'.

    pasta_trilhas : str
        Caminho da pasta contendo os arquivos .xlsx com as trilhas e cursos atualizados.

    Retorno:
    -------
    None
        A função salva a nova sheet 'Trilhas_tem_Cursos' diretamente no arquivo Excel fornecido.
    """
    # Lê os dados principais
    df_trilhas = pd.read_excel(caminho_arquivo, sheet_name='trilhas')
    df_cursos = pd.read_excel(caminho_arquivo, sheet_name='cursos')

    associacoes = []

    # Itera sobre os arquivos de trilha atualizada
    for nome_arquivo in os.listdir(pasta_trilhas):
        if not nome_arquivo.endswith(".xlsx"):
            continue

        caminho = os.path.join(pasta_trilhas, nome_arquivo)
        df = pd.read_excel(caminho)

        # Extrai os nomes da trilha e cursos
        nome_trilha = str(df['trilha'].iloc[0]).strip()

        # Recupera o id_trilha
        linha_trilha = df_trilhas[df_trilhas['nome_trilha'].str.strip() == nome_trilha]
        if linha_trilha.empty:
            print(f"❌ Trilha não encontrada: {nome_trilha}")
            continue
        id_trilha = int(linha_trilha['id_trilha'].values[0])

        for nome_curso in df['curso'].unique():
            nome_curso = str(nome_curso).strip()
            linha_curso = df_cursos[df_cursos['nome_curso'].str.strip() == nome_curso]

            if linha_curso.empty:
                print(f"⚠️ Curso não encontrado: {nome_curso}")
                continue

            id_curso = int(linha_curso['id_curso'].values[0])

            associacoes.append({
                'id_trilha': id_trilha,
                'id_curso': id_curso,
                'data_final_para_assistir': '',
                'obrigatoriedade_curso': 1
            })



    # Cria a nova sheet como DataFrame
    df_associacoes = pd.DataFrame(associacoes)

    # Lê todas as sheets e adiciona a nova
    planilhas = pd.read_excel(caminho_arquivo, sheet_name=None)
    planilhas['Trilhas_tem_Cursos'] = df_associacoes

    # Salva de volta no arquivo
    with pd.ExcelWriter(caminho_arquivo, engine='openpyxl', mode='w') as writer:
        for nome_sheet, df in planilhas.items():
            df.to_excel(writer, sheet_name=nome_sheet, index=False)

    print("✅ Sheet associativa 'Trilhas_tem_Cursos' criada com sucesso.")

#Para marcar que um usuário x esteve presente ou ausente no evento y, também serve para atualizar o registro
def registrar_participacao( caminho_arquivo, id_membro, id_evento, presenca=1): #default 1=True=Presente
    """
    Registra a participação de um membro da Feadev em um evento no arquivo Excel.

    A função abre a sheet 'membro_feadev_participa_eventos' (se existir), atualiza ou adiciona
    a participação, e salva de volta no mesmo arquivo.

    Parâmetros:
    -----------
    caminho_arquivo : str
        Caminho para o arquivo Excel que contém (ou conterá) a sheet 'membro_feadev_participa_eventos'.

    id_membro : int
        ID do membro participante.

    id_evento : int
        ID do evento.

    presenca : int, opcional (padrão=1)
        1 para presente, 0 para ausente.

    Retorno:
    --------
    None
    """
    try:
        df_participacoes = pd.read_excel(caminho_arquivo, sheet_name='membro_feadev_participa_eventos')
    except ValueError:
        # Sheet não existe ainda
        df_participacoes = pd.DataFrame(columns=['id_membro', 'id_evento', 'presença'])

    # Verifica se já existe
    existe = (
        (df_participacoes['id_membro'] == id_membro) &
        (df_participacoes['id_evento'] == id_evento)
    )

    if existe.any():
        df_participacoes.loc[existe, 'presença'] = presenca
    else:
        nova_linha = pd.DataFrame([{
            'id_membro': id_membro,
            'id_evento': id_evento,
            'presença': presenca
        }])
        df_participacoes = pd.concat([df_participacoes, nova_linha], ignore_index=True)

    # Salva no Excel
    with pd.ExcelWriter(caminho_arquivo, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df_participacoes.to_excel(writer, sheet_name='membro_feadev_participa_eventos', index=False)



# Lê o arquivo com o Banco de Dados do aplicativo MAE dev
planilhas = pd.read_excel(NOME_ARQUIVO, sheet_name=None)  # Dicionário {sheet_name: dataframe}

df_sheet_trilhas_maedev = pd.read_excel(NOME_ARQUIVO, sheet_name='trilhas')
df_sheet_cursos_maedev = pd.read_excel(NOME_ARQUIVO, sheet_name='cursos')

for trilha in LISTA_DE_TRILHAS_PARA_LER:
    trilhas_atualizadas = pd.read_excel(f'./{PASTA_TRILHAS}/{trilha}')
    df_sheet_trilhas_maedev = atualizar_trilhas_com_ids(trilhas_atualizadas, df_sheet_trilhas_maedev)
    df_sheet_cursos_maedev = atualizar_cursos_com_ids(trilhas_atualizadas, df_sheet_cursos_maedev)



# Sobrescrevendo o arquivo original
planilhas['trilhas'] = df_sheet_trilhas_maedev
planilhas['cursos'] = df_sheet_cursos_maedev

# Salva todas as sheets de volta (sobrescrevendo o arquivo original)
with pd.ExcelWriter((NOME_ARQUIVO), engine='openpyxl', mode='w') as writer:
    for nome_sheet, df in planilhas.items():
        df.to_excel(writer, sheet_name=nome_sheet, index=False)


# Associa as trilhas com os cursos
gerar_trilhas_tem_cursos(NOME_ARQUIVO, PASTA_TRILHAS)


✅ Novo curso adicionado: Applied Finance in Python (id = 0)
🎯 Atualização concluída com sucesso.
✅ Novo curso adicionado: Introduction to Portfolio Risk Management in Python (id = 0)
✅ Novo curso adicionado: Quantitative Risk Management in Python (id = 1)
✅ Novo curso adicionado: Credit Risk Modeling in Python (id = 2)
✅ Novo curso adicionado: GARCH Models in Python (id = 3)
🎯 Atualização concluída com sucesso.
✅ Novo curso adicionado: Associate Data Scientist  in Python (id = 1)
🎯 Atualização concluída com sucesso.
✅ Novo curso adicionado: Introduction to Python (id = 4)
✅ Novo curso adicionado: Intermediate Python (id = 5)
✅ Novo curso adicionado: Data Manipulation with pandas (id = 6)
✅ Novo curso adicionado: Joining Data with pandas (id = 7)
✅ Novo curso adicionado: Introduction to Statistics in Python (id = 8)
✅ Novo curso adicionado: Introduction to Data Visualization with Matplotlib (id = 9)
✅ Novo curso adicionado: Introduction to Data Visualization with Seaborn (id = 10)
✅ Nov

# Criando cookies 
Antes de puxar as trilhas do datacamp, é necessário logar e gerar os cookies

In [ ]:
from playwright.async_api import async_playwright # Rode 'playwright install' no terminal para instalar os navegadores quando for usar pela primeira vez
import pickle
import asyncio
import os
import nest_asyncio # Se estiver rodando no Jupyter ou Colab, use 'nest_asyncio' para permitir chamadas assíncronas

nest_asyncio.apply()  # permite rodar async no Jupyter/Colab

url = "https://app.datacamp.com"
cookies_path = "./cookies/datacamp_cookies.pkl"

async def save_cookies_manually():
    os.makedirs(os.path.dirname(cookies_path), exist_ok=True)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        await page.goto(url, timeout=60000)
        await context.clear_cookies()

        input("⚠️ Faça login no navegador que abriu. Depois pressione Enter aqui...")

        cookies = await context.cookies()
        with open(cookies_path, "wb") as f:
            pickle.dump(cookies, f)

        print(f"✅ Cookies salvos com sucesso em: {cookies_path}")

# Em Jupyter ou Colab:
await save_cookies_manually()

✅ Cookies salvos com sucesso em: ./cookies/datacamp_cookies.pkl


# Gerando arquivo com os cursos do datacamp
### Gera um xlsx com a ata de cursos de cada trilha dentro da lista LISTA_DE_TRILHAS_PARA_ATUALIZAR

In [ ]:


'''
Mude as urls para atualizar trilhas diferentes do datacamp, 
Para ser mais rápido, atualize uma por vez e eviter atualizar coisas desnecessárias
Funciona tanto para skill tracks quanto carrer tracks
'''
LISTA_DE_TRILHAS_PARA_ATUALIZAR = [
    #trilhas da capacitação básica 2025
    #"https://app.datacamp.com/learn/career-tracks/data-scientist-with-python",
    #'https://app.datacamp.com/learn/skill-tracks/sql-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/git-fundamentals',
    #'https://app.datacamp.com/learn/skill-tracks/excel-fundamentals',

    #trilhas da capacitação avançada 2025
    #'https://app.datacamp.com/learn/skill-tracks/time-series-with-python', #curso de Séries temporais
    #'https://app.datacamp.com/learn/skill-tracks/applied-finance-in-python', #curso de Introdução ao Gerenciamento de Risco de Portfólio em Python e curso de Garch
    #'https://app.datacamp.com/learn/skill-tracks/finance-fundamentals-in-python', #curso de Introdução à Análise de Portfólio em python
    'https://app.datacamp.com/learn/career-tracks/machine-learning-scientist-with-python' #contem boa parte dos cursos de machine learning usados
     
]
'''
CURSOS SEM TRILHAS (PRECISAM SER ADICIONADOS MANUALMENTE)
https://app.datacamp.com/learn/courses/introduction-to-optimization-in-python
https://app.datacamp.com/learn/courses/introduction-to-deep-learning-in-python
'''

nest_asyncio.apply()

cookies_path = "./cookies/datacamp_cookies.pkl"
semaphore = asyncio.Semaphore(3)

# Cria o contexto autenticado com os cookies
async def create_authenticated_context(playwright):
    if not os.path.exists(cookies_path):
        print("❌ Arquivo de cookies não encontrado. Faça o login e salve os cookies primeiro.")
        return None, None

    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()

    with open(cookies_path, "rb") as f:
        cookies = pickle.load(f)
    await context.add_cookies(cookies)

    return browser, context

# Extrai os cursos e durações de uma trilha
async def extract_courses(context, url):
    async with semaphore:
        page = await context.new_page()
        try:
            await page.goto(url, timeout=60000)
            await asyncio.sleep(random.uniform(2, 4))

            if "just a moment" not in (await page.content()).lower():
                soup = BeautifulSoup(await page.content(), "html.parser")

                track_element = soup.find("h1")
                track = track_element.get_text(strip=True) if track_element else "N/A"

                projects = {
                    optional.find_parent("div")
                    for optional in soup.select(".mfe-app-learn-hub-1moscjt")
                }

                courses = list([
                    course.get_text(strip=True)
                    for course in soup.select("h3.mfe-app-learn-hub-1yqo1j7")
                    if course.find_parent("div") not in projects
                ])
                await page.close()

                duration = {}
                for course in courses:
                    course_page = course.replace(" ", "-").lower()
                    tmp_url = f"https://app.datacamp.com/learn/courses/{course_page}"
                    page = await context.new_page()
                    try:
                        await page.goto(tmp_url, timeout=60000)
                        await asyncio.sleep(random.uniform(2, 3))

                        soup = BeautifulSoup(await page.content(), "html.parser")
                        duration_element = soup.select_one(".mfe-app-learn-hub-hdd90k")
                        duration[course] = duration_element.get_text(strip=True) if duration_element else "N/A"
                    except Exception as e:
                        print(f"⚠️ Erro ao acessar curso '{course}': {e}")
                        duration[course] = "N/A"
                    finally:
                        await page.close()

                data = [{"Trilha": track, "Curso": course, "Duração": duration[course]} for course in courses]
                print(f"✅ Cursos extraídos com sucesso da trilha: {track}")
                return data
            else:
                print("❌ Cookies expirados ou inválidos. Refaça o login.")
                return None
        except Exception as e:
            print(f"❌ Erro ao extrair curso da página {url}: {e}")
            return None

# Função principal
async def main(url_da_trilha):
    async with async_playwright() as p:
        browser, context = await create_authenticated_context(p)
        if context is None:
            return

        data = await extract_courses(context, url_da_trilha)

        await browser.close()

        if data:
            df = pd.DataFrame(data)
            df['ordem_para_assistir'] = range(len(df)) #ordem de acordo com o html da página
            df['tipo_trilha'] = 0 #0->Trilha do datacamp, 1->trilha personalizada
            df['data_final_para_assistir'] = '' #cursos do datacamp não tem datas especificas, isso só se aplica para trilhas personalizadas
            df['obrigatoriedade_curso'] = 1 #cursos do datacamp por default sao obrigatorios, opcionalidade só se aplica para trilhas personalizadas
            print(df)
            return df
        else:
            print("Nenhum dado extraído.")
            return None


# Loop principal para pegar todas as trilhas da lista
async def rodar_todas_as_trilhas():
    for trilha_do_datacamp in LISTA_DE_TRILHAS_PARA_ATUALIZAR:
        df = await main(trilha_do_datacamp)
        if df is not None:
            df.to_excel((f'../trilhas/{df.iloc[0]["Trilha"]}.xlsx'), index=False) #Salva o arquivo com titulo da trilha

#Executa a função
await rodar_todas_as_trilhas()

✅ Cursos extraídos com sucesso da trilha: Machine Learning Scientist in Python
                                  Trilha  \
0   Machine Learning Scientist in Python   
1   Machine Learning Scientist in Python   
2   Machine Learning Scientist in Python   
3   Machine Learning Scientist in Python   
4   Machine Learning Scientist in Python   
5   Machine Learning Scientist in Python   
6   Machine Learning Scientist in Python   
7   Machine Learning Scientist in Python   
8   Machine Learning Scientist in Python   
9   Machine Learning Scientist in Python   
10  Machine Learning Scientist in Python   
11  Machine Learning Scientist in Python   
12  Machine Learning Scientist in Python   
13  Machine Learning Scientist in Python   
14  Machine Learning Scientist in Python   
15  Machine Learning Scientist in Python   
16  Machine Learning Scientist in Python   
17  Machine Learning Scientist in Python   
18  Machine Learning Scientist in Python   
19  Machine Learning Scientist in Python 